# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [16]:
weather_csv = "output_data/cities.csv"

weather_df = pd.read_csv(weather_csv)

weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Hvide Sande,56.0045,8.1294,62.10,96,100,22.73,DK,1659460096
1,1,Rikitea,-23.1203,-134.9692,72.07,76,92,20.11,PF,1659460096
2,2,Butaritari,3.0707,172.7902,81.55,74,79,18.72,KI,1659460096
3,3,Clyde River,70.4692,-68.5914,41.23,100,100,12.66,CA,1659459700
4,4,Karratha,-20.7377,116.8463,68.77,64,13,15.32,AU,1659459700


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [17]:
gmaps.configure(api_key=g_key)
weather_df = weather_df.dropna()
locations = weather_df[["Lat", "Lng"]].astype(float)
humidity = weather_df["Humidity"].astype(float)

In [18]:
# Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(weather_df["Humidity"]),
                                 point_radius = 2)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [19]:
ideal_weather_df = weather_df

# A max temperature lower than 80 degrees but higher than 70
ideal_weather_df = ideal_weather_df.loc[(ideal_weather_df["Max Temp"] < 80) & (ideal_weather_df["Max Temp"] > 70)]

# Wind speed less than 10 mph
ideal_weather_df = ideal_weather_df.loc[ideal_weather_df["Wind Speed"] < 10]

# Zero Cloudiness
ideal_weather_df = ideal_weather_df.loc[ideal_weather_df["Cloudiness"] == 0]

ideal_weather_df = ideal_weather_df.dropna()

ideal_weather_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
46,46,Castro,-24.7911,-50.0119,76.41,35,0,5.06,BR,1659460104
99,99,Yatou,37.1500,122.3833,77.31,95,0,6.96,CN,1659459914
234,234,Price,39.5994,-110.8107,77.11,53,0,5.75,US,1659460039
365,365,Devils Lake,48.1128,-98.8651,75.18,76,0,8.05,US,1659460188
405,405,Pacific Grove,36.6177,-121.9166,77.38,81,0,8.01,US,1659460198


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [21]:
# Store into variable named hotel_df 

hotel_df = ideal_weather_df

# Setting parameters
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

for index,row in hotel_df.iterrows():
    # Getting latitude and longitude
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Changing Locations to look for
    params["location"] = f"{lat},{lng}"
    
    #API Url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Creating URL and API request
    print(f"Accessing results for {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("-------------------------------")
    

Accessing results for Castro.
Closest hotel is CHACARA BAILLY.
-------------------------------
Accessing results for Yatou.
Closest hotel is GreenTree Hotel Rongcheng Bus Station.
-------------------------------
Accessing results for Price.
Closest hotel is National 9 - Price River Inn.
-------------------------------
Accessing results for Devils Lake.
Closest hotel is Holiday Inn Express Devils Lake, an IHG Hotel.
-------------------------------
Accessing results for Pacific Grove.
Closest hotel is Lovers Point Inn.
-------------------------------
Accessing results for Pato Branco.
Closest hotel is Luz Hotel Pato Branco.
-------------------------------


In [22]:
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
46,46,Castro,-24.7911,-50.0119,76.41,35,0,5.06,BR,1659460104,CHACARA BAILLY
99,99,Yatou,37.1500,122.3833,77.31,95,0,6.96,CN,1659459914,GreenTree Hotel Rongcheng Bus Station
234,234,Price,39.5994,-110.8107,77.11,53,0,5.75,US,1659460039,National 9 - Price River Inn
365,365,Devils Lake,48.1128,-98.8651,75.18,76,0,8.05,US,1659460188,"Holiday Inn Express Devils Lake, an IHG Hotel"
405,405,Pacific Grove,36.6177,-121.9166,77.38,81,0,8.01,US,1659460198,Lovers Point Inn
413,413,Pato Branco,-26.2286,-52.6706,75.29,52,0,5.46,BR,1659460200,Luz Hotel Pato Branco


In [24]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [25]:
# Add marker layer ontop of heat map
# Assign the marker layer to a variable
locations = hotel_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(locations)
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)

# Display figure
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig


Figure(layout=FigureLayout(height='420px'))